In [1]:
# Author: Andrew Louwagie Gordon
# Date Created: 29May2018
# Last Modified: 08Jun2018
#
# Hacked on by Juan Cabanela to get color index calculations working.

In [2]:
# Import Block
# Import the necessary packages
import ipywidgets as widgets
import bqplot as bq
import numpy as np
import tempNcolor as tc
import number_formatting as nf
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import pythreejs as p3j

In [3]:
# Functions Definitions Block
# Define the constants
c = 3e8
h = 6.626e-34
k = 1.38e-23

def Wein(T):
    '''
    This is Wein's Law and returns the peak wavelength.
    '''
    lamda_max = 0.002897755 / T
    return lamda_max

def blackbody(lamda, T):
    global c, h, k
    '''
    This function takes the array of wavelengths and the temperature from the slider and returns an array of fluxes that
    correspond to the blackbody curve.
    '''
    return ( (2 * h * (c ** 2)) / (lamda ** 5) ) / ( np.exp((h * c) / (lamda * k * T)) - 1 )

def cr(change=None):
    '''
    This function updates the first figure which gives the blackbody of the model star and shows the filter curves.
    '''
    global wavelengths
    spectrum = blackbody(wavelengths,Temp.value)
    Blackbody.y = [spectrum]
    pw = Wein(Temp.value)
    pwlist = nf.exp2LaTeX(pw) # Get LaTeX form of number
    peak_wavelength.value =  r'\({}\)'.format(pwlist[1]) # Write LaTeX form
    
    counts = tc.temp2rgb(Temp.value) # Convert temperature into rgb counts
    hex_color = tc.rgb2hex(counts) # Convert the rgb counts into a hexidecimal value that reflects the color of the star
    star.material.color = hex_color[0]
    
    Model_Star.y = [blackbody(wavelengths,Temp.value)]
    
    if plotted_star.index == 0:
        star_fig.marks = [Proxima_Centauri, Model_Star]
        etemp = 3300
    elif plotted_star.index == 1:
        star_fig.marks = [Pollux, Model_Star]
        etemp = 4700
    elif plotted_star.index == 2:
        star_fig.marks = [Sun, Model_Star]
        etemp = 5800
    elif plotted_star.index == 3:
        star_fig.marks = [Iota_Piscium, Model_Star]
        etemp = 6300
    elif plotted_star.index == 4:
        star_fig.marks = [Polaris, Model_Star]
        etemp = 7200
    elif plotted_star.index == 5:
        star_fig.marks = [Deneb, Model_Star]
        etemp = 8500
    
    star_pw_f = blackbody(Wein(etemp),etemp)
    pw_f = blackbody(Wein(Temp.value),Temp.value)
    
    if star_pw_f >= pw_f:
        y_sc.max = star_pw_f
    else:
        y_sc.max = pw_f
    
    # Update the flux magnitudes and color values
    (U_mag, B_mag, V_mag, R_mag, I_mag) = compute_mag(wavelengths, spectrum, V_calibration=5, debug=False) 
    UB_color = U_mag - B_mag
    BV_color = B_mag - V_mag
    VR_color = B_mag - R_mag
    RI_color = R_mag - I_mag
    U_mag_report.value = str(round(U_mag, 1))
    B_mag_report.value = str(round(B_mag, 1))
    V_mag_report.value = str(round(V_mag, 1))
    R_mag_report.value = str(round(R_mag, 1))
    I_mag_report.value = str(round(I_mag, 1))
    UB_color_report.value = str(round(UB_color, 1))
    BV_color_report.value = str(round(BV_color, 1))
    VR_color_report.value = str(round(VR_color, 1))
    RI_color_report.value = str(round(RI_color, 1))
    
    # Update the filter curves to fit the blackbody 
    (U_spec, B_spec, V_spec, R_spec, I_spec) = compute_UBVRIspectra(wavelengths, spectrum, debug=False)
    U_line.y = [U_spec]
    B_line.y = [B_spec]
    V_line.y = [V_spec]
    R_line.y = [R_spec]
    I_line.y = [I_spec]
    
def fini(change=None):
    '''
    This function updates the second figure which shows the peak of the function with respect to the visible spectrum.
    '''
    global my_wl, wide_line
    my_f = blackbody(wavelengths,Temp.value) 
    y_zeros = np.zeros_like(wavelengths)
    y_array = np.array([y_zeros, my_f])
    fin_y_array = y_array.transpose()    
    wide_line.y = [fin_y_array]   

# The following functions are the intelectual property of Juan Cabanela
def create_functions(debug=False):
    global Uspline, Bspline, Vspline, Rspline, Ispline

    ###
    ### The U, B, V, R filter transmission data was downloaded from the SNCosmos source repository at
    ### https://github.com/sncosmo/sncosmo/tree/master/sncosmo/data/bandpasses/bessell
    ### which are based on the work of Michael Bessell (Bessell, M.S. 1990, PASP, 102, 1181)
    ### to reproduce the classic Johnson-Cousins passbands using cheap optical glass filters.
    ###

    U = {300.0:0.000, 305.0:0.016, 310.0:0.068, 315.0:0.167, 320.0:0.287,
         325.0:0.423, 330.0:0.560, 335.0:0.673, 340.0:0.772, 345.0:0.841,
         350.0:0.905, 355.0:0.943, 360.0:0.981, 365.0:0.993, 370.0:1.000,
         375.0:0.989, 380.0:0.916, 385.0:0.804, 390.0:0.625, 395.0:0.423,
         400.0:0.238, 405.0:0.114, 410.0:0.051, 415.0:0.019, 420.0:0.000}

    B = {360.0:0.000, 370.0:0.030, 380.0:0.134, 390.0:0.567, 400.0:0.920,
         410.0:0.978, 420.0:1.000, 430.0:0.978, 440.0:0.935, 450.0:0.853,
         460.0:0.740, 470.0:0.640, 480.0:0.536, 490.0:0.424, 500.0:0.325,
         510.0:0.235, 520.0:0.150, 530.0:0.095, 540.0:0.043, 550.0:0.009,
         560.0:0.000}

    V = {470.0:0.000, 480.0:0.030, 490.0:0.163, 500.0:0.458, 510.0:0.780,
         520.0:0.967, 530.0:1.000, 540.0:0.973, 550.0:0.898, 560.0:0.792,
         570.0:0.684, 580.0:0.574, 590.0:0.461, 600.0:0.359, 610.0:0.270,
         620.0:0.197, 630.0:0.135, 640.0:0.081, 650.0:0.045, 660.0:0.025,
         670.0:0.017, 680.0:0.013, 690.0:0.009, 700.0:0.000}

    R = {550.0:0.00, 560.0:0.23, 570.0:0.74, 580.0:0.91, 590.0:0.98, 600.0:1.00,
         610.0:0.98, 620.0:0.96, 630.0:0.93, 640.0:0.90, 650.0:0.86, 660.0:0.81,
         670.0:0.78, 680.0:0.72, 690.0:0.67, 700.0:0.61, 710.0:0.56, 720.0:0.51,
         730.0:0.46, 740.0:0.40, 750.0:0.35, 800.0:0.14, 850.0:0.03, 900.0:0.00}

    I = {700.0:0.000, 710.0:0.024, 720.0:0.232, 730.0:0.555, 740.0:0.785,
         750.0:0.910, 760.0:0.965, 770.0:0.985, 780.0:0.990, 790.0:0.995,
         800.0:1.000, 810.0:1.000, 820.0:0.990, 830.0:0.980, 840.0:0.950,
         850.0:0.910, 860.0:0.860, 870.0:0.750, 880.0:0.560, 890.0:0.330,
         900.0:0.150, 910.0:0.030, 920.0:0.000}

    # Get wavelengths and tranmission values as arrays (for better interpolation)
    U_wavelength = np.array([k for k in U.keys()])/1e9
    B_wavelength = np.array([k for k in B.keys()])/1e9
    V_wavelength = np.array([k for k in V.keys()])/1e9
    R_wavelength = np.array([k for k in R.keys()])/1e9
    I_wavelength = np.array([k for k in I.keys()])/1e9
    U_trans = np.array([v for v in U.values()])
    B_trans = np.array([v for v in B.values()])
    V_trans = np.array([v for v in V.values()])
    R_trans = np.array([v for v in R.values()])
    I_trans = np.array([v for v in I.values()])

    # Create functions using interpolation to allow filter tranmittance from wavelength (in meters)
    # [any attempt to go outside the bounds gives a zero for the filter tranmission] 
    Uspline = interp1d(U_wavelength, U_trans, kind='cubic', bounds_error=False, fill_value = (0,0))
    Bspline = interp1d(B_wavelength, B_trans, kind='cubic', bounds_error=False, fill_value = (0,0))
    Vspline = interp1d(V_wavelength, V_trans, kind='cubic', bounds_error=False, fill_value = (0,0))
    Rspline = interp1d(R_wavelength, R_trans, kind='cubic', bounds_error=False, fill_value = (0,0))
    Ispline = interp1d(I_wavelength, I_trans, kind='cubic', bounds_error=False, fill_value = (0,0))
    
    if (debug):
        # Determine wavelength ranges to plot for (delete in final code)
        max_wavelength_nm = 3000  # maximum wavelength to compute for in nm
        wavelengths_nm = np.arange(start = 0, stop = max_wavelength_nm+2, step=5)
        wavelengths_m = wavelengths_nm/1e9

        ## Plot up the fits (note that these are functions now, so I can use same wavelength range for all)
        ##  (delete in final code)
        plt.plot(U_wavelength*1e9, U_trans, color='violet', marker="o", linestyle="none", label="U (actual)")
        plt.plot(B_wavelength*1e9, B_trans, color='blue', marker="o", linestyle="none", label="B (actual)")
        plt.plot(V_wavelength*1e9, V_trans, color='green', marker="o", linestyle="none", label= "V (actual)")
        plt.plot(R_wavelength*1e9, R_trans, color='red', marker="o", linestyle="none", label= "R (actual)")
        plt.plot(I_wavelength*1e9, I_trans, color='black', marker="o", linestyle="none", label= "I (actual)")
        plt.plot(wavelengths_nm, Uspline(wavelengths_m), color='violet', label="U (interpolation)")
        plt.plot(wavelengths_nm, Bspline(wavelengths_m), color='blue', label="B (interpolation)")
        plt.plot(wavelengths_nm, Vspline(wavelengths_m), color='green', label="V (interpolation)")
        plt.plot(wavelengths_nm, Rspline(wavelengths_m), color='red', label="R (interpolation)")
        plt.plot(wavelengths_nm, Ispline(wavelengths_m), color='black', label="I (interpolation)")
        plt.xlim(0,1500)
        plt.legend()
        plt.show()

    ## At this point, I have defined the 5 functions to allow estimation of the transmission at any 
    ## wavelength through any of the Cousin-Johnson filters as global variables.
    return

def compute_UBVRIspectra(wavelengths, spectrum, debug=False):
    global Uspline, Bspline, Vspline, Rspline, Ispline
    
    '''
    This function will compute U, B, V, R, and I spectra, that is given the raw spectrum, how much gets through the 
    individual filters, is used to plot the data.
    '''
    
    # Check if the spectrum is acceptable, and if so, check that interpolation functions are defined.
    if (np.isfinite(spectrum).all()):
        # Check that the tranmission functions are defined by checking if Uspline is defined
        try:
            Uspline(500)
        except NameError:
            create_functions(debug=debug)
    else:
        raise ValueError('compute_mag: Your spectrum contains inf, death!')

    # Compute spectra getting through each filter
    U_spectrum = spectrum*Uspline(wavelengths)
    B_spectrum = spectrum*Bspline(wavelengths)
    V_spectrum = spectrum*Vspline(wavelengths)
    R_spectrum = spectrum*Rspline(wavelengths)
    I_spectrum = spectrum*Ispline(wavelengths)

    ##
    ## Show spectrum getting through each filter versus original unfiltered spectrum
    ## (disable this in final code)
    ##
    if (debug):
        plt.plot(wavelengths*1e9, spectrum, color="orange", label="5800K Spectrum")
        plt.plot(wavelengths*1e9, U_spectrum, color='violet', label="U profile")
        plt.plot(wavelengths*1e9, B_spectrum, color='blue', label="B profile")
        plt.plot(wavelengths*1e9, V_spectrum, color='green', label="V profile")
        plt.plot(wavelengths*1e9, R_spectrum, color='red', label="R profile")
        plt.plot(wavelengths*1e9, I_spectrum, color='black', label="I profile")
        plt.legend()
        plt.show()
    
    return (U_spectrum, B_spectrum, V_spectrum, R_spectrum, I_spectrum)

def compute_mag(wavelengths, spectrum, V_calibration = 7.0, debug=False):
    global Uspline, Bspline, Vspline, Rspline, Ispline
    
    '''
    This function will compute U, B, V, R, and I magnitudes for the spectrum you pass it.  The magnitudes are UNCALIBRATED
    and so they should NOT be displayed to the user, however, they are on the same system, so they are appropriate for 
    color index calculation.  However, if you want to fake it, I set it up so they are all scaled so V = V_calbration.
    '''
    
    # Check if the spectrum is acceptable, and if so, check that interpolation functions are defined.
    if (np.isfinite(spectrum).all()):
        # Check that the tranmission functions are defined by checking if Uspline is defined
        try:
            Uspline(500)
        except NameError:
            create_functions()
    else:
        raise ValueError('compute_mag: Your spectrum contains inf, death!')
    
    ##
    ## Show the filter transmission and spectrum on same scale
    ## (disable this in final code)
    ##
    if (debug):
        plt.plot(wavelengths*1e9, spectrum/spectrum.max(), color="orange", label="5800K Spectrum")
        plt.plot(wavelengths*1e9, Uspline(wavelengths), color='violet', label="U (interpolation)")
        plt.plot(wavelengths*1e9, Bspline(wavelengths), color='blue', label="B (interpolation)")
        plt.plot(wavelengths*1e9, Vspline(wavelengths), color='green', label="V (interpolation)")
        plt.plot(wavelengths*1e9, Rspline(wavelengths), color='red', label="R (interpolation)")
        plt.plot(wavelengths*1e9, Ispline(wavelengths), color='black', label="I (interpolation)")
        plt.legend()
        plt.show()

    # Compute spectra getting through each filter
    (U_spectrum, B_spectrum, V_spectrum, R_spectrum, I_spectrum) = compute_UBVRIspectra(wavelengths, spectrum, debug=debug)

    # Compute the cumilative fluxes through each filter and the colors
    # (These lines are only to check the linear flux values and can be dropped)
    if (debug):
        U_flux = U_spectrum.sum()
        B_flux = B_spectrum.sum()
        V_flux = V_spectrum.sum()
        R_flux = R_spectrum.sum()
        I_flux = I_spectrum.sum()
        print ("U, B, V, R, I flux: ", U_flux, B_flux, V_flux, R_flux, I_flux)

    # Uncalibrated magnitudes reset so V_mag value is set to V_calibrated
    V_mag = -2.5*np.log10(V_spectrum.sum())
    delta = V_mag - V_calibration
    V_mag -= delta
    U_mag = -2.5*np.log10(U_spectrum.sum()) - delta
    B_mag = -2.5*np.log10(B_spectrum.sum()) - delta
    R_mag = -2.5*np.log10(R_spectrum.sum()) - delta
    I_mag = -2.5*np.log10(I_spectrum.sum()) - delta

    return (U_mag, B_mag, V_mag, R_mag, I_mag)

def ConfigStars(r, t):
    '''
    Determines the radii (in solar radii), temperature (in K), and hexcolor of the two stars assuming 
    they are main sequence stars and returns that information.
    '''
    
    # Determine approximate radius in solar radii for both stars.
    radius1= r

    # Determines the approximate temperature of each star.
    temp1 = t
    
    # Use scalar temperature to estimate hexcolor appropriate to each star
    hexcolor1 = tc.rgb2hex(tc.temp2rgb(temp1))[0]
    
    return (radius1, temp1, hexcolor1)

In [4]:
# Variable Definition Block

# Define the peak wavelength for initial calculations and plots
pw = Wein(2800) 

# Define all wavelengths from 1 nm to 1600 nm
wavelengths = np.arange(1.0e-9, 1601.0e-9, 1e-9)
wavelengths_m = wavelengths/(1e9)

pwlist = nf.exp2LaTeX(pw) # Make a list from the number2LaTeX converter being used

spectrum = blackbody(wavelengths, 2800) # Generate an initial spectrum to be used in the early calculations

# Calculate the initial U, B, V, R, I magnitudes and spectra, activate debug to plot for early visual plot otherwise
#    values will be plotted later
(U_mag, B_mag, V_mag, R_mag, I_mag) = compute_mag(wavelengths, spectrum, V_calibration=5, debug=False)
(U_spec, B_spec, V_spec, R_spec, I_spec) = compute_UBVRIspectra(wavelengths, spectrum, debug=False)

# Get the values for the color
UB_color = U_mag - B_mag
BV_color = B_mag - V_mag
VR_color = B_mag - R_mag
RI_color = R_mag - I_mag

In [5]:
# Widget Definitions Block
# This widget controls the temperature of the black body
Temp = widgets.FloatSlider(
    min=2800,
    max=42000,
    step=100,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

# Widget to report updated peak wavelength 
peak_wavelength = widgets.Label(
    value = r'\({}\)'.format(pwlist[1]),
    placeholder = 'Type something',
    disabled = True   
)

# These widgets report the U, B, V, R, I magnitudes and spectra that are calculated based on temperature 
U_mag_report = widgets.Label(
    value = str(round(U_mag, 1)),
    placeholder = 'Type something',
    disabled = True   
)

B_mag_report = widgets.Label(
    value = str(round(B_mag, 1)),
    placeholder = 'Type something',
    disabled = True   
)

V_mag_report = widgets.Label(
    value = str(round(V_mag, 1)),
    placeholder = 'Type something',
    disabled = True   
)

R_mag_report = widgets.Label(
    value = str(round(R_mag, 1)),
    placeholder = 'Type something',
    disabled = True   
)

I_mag_report = widgets.Label(
    value = str(round(I_mag, 1)),
    placeholder = 'Type something',
    disabled = True   
)

UB_color_report = widgets.Label(
    value = str(round(UB_color, 1)),
    placeholder = 'Type something',
    disabled = True   
)

BV_color_report = widgets.Label(
    value = str(round(BV_color, 1)),
    placeholder = 'Type something',
    disabled = True   
)

VR_color_report = widgets.Label(
    value = str(round(VR_color, 1)),
    placeholder = 'Type something',
    disabled = True   
)

RI_color_report = widgets.Label(
    value = str(round(RI_color, 1)),
    placeholder = 'Type something',
    disabled = True   
)

plotted_star = widgets.RadioButtons(
    options=['Proxima Centauri', 'Pollux', 'Sun', 'Iota Piscium', 'Polaris', 'Deneb'],
    #description='Star:',
    disabled=False
)

In [6]:
# Required Data Block
# Update peak wavelength
my_f = blackbody(wavelengths,Temp.value)

In [7]:
# Figure 1 Definition Block
# Define the scale
x_sc1 = bq.LinearScale()
y_sc1 = bq.LinearScale()

# Define the axes
ax_x = bq.Axis(label='Wavelength (m)', scale=x_sc1, tick_format='0.0e-9f')
ax_y = bq.Axis(label='Flux', scale=y_sc1, orientation='vertical', tick_format='0.0e9f')

# Define the markers which are the blackbody and the filter curves
Blackbody = bq.Lines(x=wavelengths, 
             y=spectrum,
             name = ['Model Blackbody'],
             scales={'x': x_sc1, 'y': y_sc1},
             display_legend=True,
             colors=['orange'], 
             labels=['Model Star'])

U_line = bq.Lines(x=wavelengths,
             y=U_spec,
             name = ['U Profile'],
             scales={'x': x_sc1, 'y': y_sc1},
             display_legend=True,
             fill = 'inside',
             fill_colors = ['violet'],
             opacities = [0.5],
             colors=['violet'], 
             labels=['U Profile'])

B_line = bq.Lines(x=wavelengths,
             y=B_spec,
             name = ['B Profile'],
             scales={'x': x_sc1, 'y': y_sc1},
             display_legend=True,
             fill = 'inside',
             fill_colors = ['blue'],
             opacities = [0.5],
             colors=['blue'], 
             labels=['B Profile'])

V_line = bq.Lines(x=wavelengths, 
             y=V_spec,
             name = ['c'],
             scales={'x': x_sc1, 'y': y_sc1},
             display_legend=True,
             fill = 'inside',
             fill_colors = ['green'],
             opacities = [0.5],
             colors=['green'], 
             labels=['V Profile'])

R_line = bq.Lines(x=wavelengths, 
             y=R_spec,
             name = ['R Profile'],
             scales={'x': x_sc1, 'y': y_sc1},
             display_legend=True,
             fill = 'inside',
             fill_colors = ['red'],
             opacities = [0.5],
             colors=['red'], 
             labels=['R Profile'])

I_line = bq.Lines(x=wavelengths,
             y=I_spec,
             name = ['I Profile'],
             scales={'x': x_sc1, 'y': y_sc1},
             display_legend=True,
             fill = 'inside',
             fill_colors = ['black'],
             opacities = [0.5],
             colors=['black'], 
             labels=['I Profile'])

# Call the function to update the figure in real time
Temp.observe(cr, names=['value'])

# Define Figure 1
fig = bq.Figure(title = 'Blackbody with Color Filters',axes=[ax_x, ax_y], animation = 1000, 
                marks=[Blackbody, U_line, B_line, V_line, R_line, I_line],
                legend_location='top-right', legend_style={'fill': 'white'})

In [8]:
# Figure 2 Definition Block
# Define the scale
x_sc2 = bq.LinearScale()
y_sc2 = bq.LinearScale()

# Define the axes
ax_x2 = bq.Axis(label='Wavelength (m)', scale=x_sc1, tick_format='0.0e-9f')
ax_y2 = bq.Axis(label='Flux', scale=y_sc1, orientation='vertical', tick_format='0.0e9f')

# This code define the colors to be plotted and which wavelengths they are plotted at, uses hexidecimal designation
colors_array = tc.wav2hex(wavelengths*10**9)
colors_list = colors_array.tolist()

# The curve for this figure is created by drawing a bunch of vertical lines that go from zero to the blackbody curve, these 
#     arrays provide the proper pairs of points that define each individual line
x_array = np.array([wavelengths, wavelengths])
fin_x_array = x_array.transpose() # Arrays must be transposed to get pairs of numbers
y_zeros = np.zeros_like(wavelengths)
y_array = np.array([y_zeros, my_f])
fin_y_array = y_array.transpose() # Arrays must be transposed to get pairs of numbers

# This is the line command that draws all the lines
wide_line = bq.Lines(x = fin_x_array, y = fin_y_array, scales={'x': x_sc2, 'y': y_sc2}, colors=colors_list)

# Implement the update function
Temp.observe(fini, names=['value'])

# Define Figure 2
fig2 = bq.Figure(title = 'Blackbody Peak Wavelength', axes=[ax_x2, ax_y2], animation = 1000, marks = [wide_line])

In [9]:
# Set scale factor for radius (10 pixels per solar radius)
scale_factor = 1

# Set viewer size
view_width = 400
view_height = 400

# Set initial parameters based on stellar parameters
(radius1, temp1, hexcolor1) = ConfigStars(5, Temp.value)
r1 = scale_factor*radius1

# Save initial radius to scale all other radii to this
init_r = r1

# set the scale
scale1 = (r1/init_r, r1/init_r, r1/init_r)

# Create sphere using a texture based on public domain STEREO Heliographic map made with 
# combined data from STEREO-Ahead, STEREO-Behind and SDO from Dec. 30, 2011.  Image downloaded from
#    https://stereo.gsfc.nasa.gov/360blog/ 
# and rescaled.
sunTexture = p3j.ImageTexture(imageUri='images/sun_surface.jpg')
star = p3j.Mesh(geometry=p3j.SphereBufferGeometry(radius=r1, widthSegments=64, heightSegments=32),
                material=p3j.MeshBasicMaterial(color=hexcolor1, map=sunTexture),
                position=[0, 0, 0], scale=scale1)

# Define viewing region size
xmax=1.5*10

# Makes the scene environment, not sure how the background works yet
scene2 = p3j.Scene(children=[star, p3j.AmbientLight(color='white')], background='black')

# Creates the camera so you can see stuff.  Place the cemera just above the x-axis and orient camera so up
# is along y-axis.
starcam = p3j.PerspectiveCamera(position=[2*xmax, 0.1*xmax, 0], up=[0, 1, 0],
                      children=[p3j.DirectionalLight(color='white', 
                                                 position=[xmax, xmax, xmax], 
                                                 intensity=0.5)])
# Makes a controller to use for the 
controller = p3j.OrbitControls(controlling=starcam, enableRotate=True, enableZoom=False)

# creates the object that gets displayed to the screen
renderer2 = p3j.Renderer(camera=starcam, 
                    scene=scene2, 
                    controls=[controller],
                    width=view_width, height=view_height)

box_layout = widgets.Layout(align_items='center', justify_content = 'flex-end', border='none', width='100%')

fig3 = widgets.VBox([widgets.Label ("Model Star:"), renderer2], layout = box_layout)

In [10]:
# Figure 1 Definition Block
# Define the scale
x_sc = bq.LinearScale()
y_sc = bq.LinearScale(max = blackbody(Wein(3300),3300))

# Define the axes
ax_x = bq.Axis(label='Wavelength (m)', scale=x_sc, tick_format='0.0e-9f')
ax_y = bq.Axis(label='Flux', scale=y_sc, orientation='vertical', tick_format='0.0e9f')

# Define the Markers
Sun = bq.Lines(x = wavelengths,
             y = blackbody(wavelengths,5800),
             name = ['5800 K'],
             display_legend=True,
             scales={'x': x_sc, 'y': y_sc},
             colors=['yellow'],
             labels=['The Sun'])

Proxima_Centauri = bq.Lines(x = wavelengths,
             y = blackbody(wavelengths,3300),
             name = ['3300 K'],
             display_legend=True,
             scales={'x': x_sc, 'y': y_sc},
             colors=['red'],
             labels=['Proxima Centauri'])

Pollux = bq.Lines(x = wavelengths,
             y = blackbody(wavelengths,4700),
             name = ['4700 K'],
             display_legend=True,
             scales={'x': x_sc, 'y': y_sc},
             colors=['orange'],
             labels=['Pollux'])

Iota_Piscium = bq.Lines(x = wavelengths,
             y = blackbody(wavelengths,6300),
             name = ['6300 K'],
             display_legend=True,
             scales={'x': x_sc, 'y': y_sc},
             colors=['blue'],
             labels=['Iota Piscium'])

Polaris = bq.Lines(x = wavelengths,
             y = blackbody(wavelengths,7200),
             name = ['7200 K'],
             display_legend=True,
             scales={'x': x_sc, 'y': y_sc},
             colors=['#9acd32'], # yellow-green
             labels=['Polaris'])

Deneb = bq.Lines(x = wavelengths,
             y = blackbody(wavelengths,8500),
             name = ['8500 K'],
             display_legend=True,
             scales={'x': x_sc, 'y': y_sc},
             colors=['#00ffff'], # cyan
             labels=['Deneb'])

Model_Star = bq.Lines(x = wavelengths,
             y = blackbody(wavelengths,2800),
             name = ['Model'],
             display_legend=True,
             scales={'x': x_sc, 'y': y_sc},
             colors=['#ff00ff'],#7851a9 #4b0082
             labels=['Model Star'])

# Call the function to update the figure in real time
Temp.observe(cr, names=['value'])
plotted_star.observe(cr, names=['index'])

# Define Figure 1
star_fig = bq.Figure(title = 'Common Stars and their Curves',axes=[ax_x, ax_y], animation = 1000, 
                marks=[Proxima_Centauri, Model_Star], 
                legend_location='top-right', legend_style={'fill': 'white'})

In [11]:
#star_fig
temp_slide = widgets.HBox([widgets.Label('Temperature (K):'),Temp])
star_selector = widgets.HBox([widgets.Label('Stars:'),plotted_star])
BOX = widgets.HBox([star_fig, fig2], layout = widgets.Layout(align_items = 'center'))
BOX.children[0].layout.width = '450px' # Resize Figure 1 so that both figures fit on the screen
BOX.children[1].layout.width = '450px' # Resize Figure 2 so that both figures fit on the screen
selectors = widgets.HBox([star_selector, temp_slide], layout = widgets.Layout(align_items = 'center'))
fig_box = widgets.VBox([BOX, selectors], layout = widgets.Layout(align_items = 'center'))

fig_box

In [12]:
# Display Block
# Label the all the widgets to prevent text cut-off
p_wave = widgets.HBox([widgets.Label('Peak Wavelength (m):'), peak_wavelength])
p_wave.children[1].layout.width = '100px' # Make this widget large enough to handle the LaTeX notation
#umag = widgets.HBox([widgets.Label('U-Magnitude:'), U_mag_report])
#bmag = widgets.HBox([widgets.Label('B-Magnitude:'), B_mag_report])
#vmag = widgets.HBox([widgets.Label('V-Magnitude:'), V_mag_report])
#rmag = widgets.HBox([widgets.Label('R-Magnitude:'), R_mag_report])
#imag = widgets.HBox([widgets.Label('I-Magnitude:'), I_mag_report])
ubcol =widgets.HBox([widgets.Label('U-B Color:'), UB_color_report])
ubcol.children[1].layout.width = '100px'
bvcol =widgets.HBox([widgets.Label('B-V Color:'), BV_color_report])
bvcol.children[1].layout.width = '100px'
vrcol =widgets.HBox([widgets.Label('V-R Color:'), VR_color_report])
vrcol.children[1].layout.width = '100px'
ricol =widgets.HBox([widgets.Label('R-I Color:'), RI_color_report])
ricol.children[1].layout.width = '100px'

# Make the figures the focus of the screen
top = widgets.HBox([fig,fig3]) # Make the visual figures appear on the top for eay veiwing
top.children[0].layout.width = '450px' # Resize Figure 1 so that both figures fit on the screen
top.children[1].layout.width = '450px' # Resize Figure 2 so that both figures fit on the screen
top.children[1].layout.height = '450px' # Resize Figure 2 so that both figures fit on the screen

# Assign peak wavelength and the temperature slider to sit under the the figures
bottom = widgets.HBox([temp_slide,p_wave])

# Make the magnitude display widgets stand vertically on the side
middle = widgets.HBox([ubcol, bvcol, vrcol, ricol])

# Organize the widgets to display in a presentable fashion 
box = widgets.VBox([top, middle, bottom], layout = widgets.Layout(align_items = 'center'))
big_box = widgets.HBox([box], layout = widgets.Layout(align_items = 'center'))
big_box # Display it